<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/region_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install cityhash
import cityhash

from copy import deepcopy

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for cityhash: filename=cityhash-0.2.3.post9-cp36-cp36m-linux_x86_64.whl size=82877 sha256=7e3994199aa024bea31f4278565bf683057cc414175269ef0e4eb1e9085d03a8
  Stored in directory: /root/.cache/pip/wheels/a4/4d/ae/cf0f1f8058ca2738fe972c69291083a0a06765271fb1047edd
Successfully built cityhash


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
org_info = pd.read_csv('/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/organizations.csv')

In [ ]:
import collections

# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']

# Select organization data
# Clean the df

org_info_selected = org_info[['uuid', 'region']]
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)

regions = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  region = row['region']  
  regions[region] += 1

# Filter low count regions
low_count_threshold = 15
keys_to_remove = []
for region, count in regions.items():
  if count < low_count_threshold:
    keys_to_remove.append(region)

for key in keys_to_remove:
  del regions[key]

# Create index to word mapping
region_to_index = {}
region_index = 0
for region, count in regions.items():
  region_to_index[region] = region_index
  region_index += 1

print(region_to_index)
region_size = region_index

Processing  842699  rows.
100000  of  842699
200000  of  842699


In [11]:
from scipy.sparse import coo_matrix
import collections

# Load our filtered region dictionary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  value = row['region']
  if value in regions:
    row_indices.append(i)
    col_indices.append(region_to_index[value])
    data_values.append(1)

unclustered = coo_matrix((data_values, (row_indices, col_indices)))
print(unclustered.toarray().shape)

Processing  842699  rows.


KeyError: ignored

In [10]:
import scipy.sparse

companies = org_info_selected['uuid']

lookup_index = []
lookup_value = []

used_hashes = set()

row_indices = []
col_indices = []
data_values = []
for (i, row) in companies.iteritems():
  uuid = row
  hash = cityhash.CityHash64(uuid)

  if hash in used_hashes:  # Hash collision!
    print(hash)
    continue          # which shouldn't really happen
  used_hashes.add(hash)

  lookup_value.append(uuid)
  lookup_index.append(hash)

  row_indices.append(i)
  col_indices.append(0)
  data_values.append(hash)

hashed_uuids = coo_matrix((data_values, (row_indices, col_indices)))
print(np.shape(unclustered))
print(np.shape(hashed_uuids))
joined = scipy.sparse.hstack([hashed_uuids, unclustered])
  
## Generate company lookup
lookup = pd.DataFrame.from_dict({'uuid':lookup_index, 'hash':data_values})
lookup.to_csv('/lookup.csv')

category_features = pd.DataFrame.sparse.from_spmatrix(joined)
category_features.to_pickle('/region_features.pkl')



NameError: ignored